In [70]:
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [71]:
# Set random seed
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
dde.backend.tf.random.set_random_seed(seed)

# Set hyperparameters
n_output = 3 # postition (x), theta, force on cart (u_norm)

num_domain = 1000

n_adam = 5000

lr = 2e-2 # for Adam
loss_weights = [1. for _ in range(8)]

# Set physical parameters
tmin, tmax = 0.0, 10.0
xmin, xmax = -5.0, 5.0
target = -1. # cos(theta) should be close to 180 degrees

# Define constants
M = 0.5 # Mass of the cart
m = 0.2 # Mass of the pendulum
b = 0.1 # Friction coefficient
I = 0.006 # Moment of inertia
g = 9.8 # Gravity
l = 0.3 # Length of the pendulum
u_max = 10  # Maximum force

In [72]:
class Custom_BC(dde.icbc.BC):
    def __init__(self, geom, func, on_boundary, component=0):
        super().__init__(geom, on_boundary, component)
        self.func = dde.icbc.boundary_conditions.npfunc_range_autocache(dde.utils.return_tensor(func))
        
    def error(self, X, inputs, outputs, beg, end, aux_var=None):
        # beg and end specify the current batch range
        values = self.func(X, beg, end, aux_var)
        theta = outputs[:, 0:1]
        goal = tf.cos(theta)
        return goal[beg:end, self.component:self.component + 1] - values

In [73]:
def ode(t, u):
    x, theta, force = u[:, 0:1], u[:, 1:2], u[:, 2:3]
    x_dot = dde.grad.jacobian(x, t)
    theta_dot = dde.grad.jacobian(theta, t)
    x_ddot = dde.grad.jacobian(x_dot, t)
    theta_ddot = dde.grad.jacobian(theta_dot, t)

    denominator = I * (M + m) + M * m * l * l

    res1 = x_ddot - (((-(I + m * l * l) * b * x_dot) + (m * m * g * l * l * theta) + ((I + m * l * l) * force)) / denominator)
    res2 = theta_ddot - (((-m * b * l * theta_dot) + (m * g * (M + m) * l * theta) +( m * l * force)) / denominator)

    return [res1, res2]

In [74]:
def initial(_, on_initial):
    return on_initial

def boundary_left(t, on_boundary):
    '''
        on_boundary is passed here by deepxde and it serves as an intial filter which tells if a point lies on the boundary or not

        np.isclose(t[0], tmin) checks if the point is on the left boundary or not and this is a second filter
    '''

    return on_boundary * np.isclose(t[0], tmin)

def boundary_right(t, on_boundary):
    '''
        on_boundary is passed here by deepxde and it serves as an intial filter which tells if a point lies on the boundary or not

        np.isclose(t[0], tmax) checks if the point is on the right boundary or not and this is a second filter
    '''

    return on_boundary * np.isclose(t[0], tmax)

In [75]:
geom = dde.geometry.TimeDomain(tmin, tmax)

# INITIAL CONDITIONS
position_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=0) # posittion = 0 at time = 0
theta_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=1) # theta = 0 at time = 0
force_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=2) # force = 0 at time = 0

# NEUMANN CONDITIONS
velocity_initial = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=0) # cart velocity = 0 at time = 0
angular_velocity_initial = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=1) # angular velocity 1 = 0 at time = 0

# CUSTOM BOUNDARY CONDITIONS - GOAL AND POSITION RANGE
goal = Custom_BC(geom, lambda t: np.array([target]), boundary_right) # custom ICBC

losses = [position_initial, theta_initial, force_initial, velocity_initial, angular_velocity_initial, goal]

data = dde.data.PDE(geom, ode, losses, num_domain=num_domain, num_boundary=2)
# dataset size here will be 1002 (1000 domain + 2 boundary)

In [76]:
net = dde.nn.FNN([1] + [64] * 3 + [n_output], "tanh", "Glorot normal")

In [77]:
resampler = dde.callbacks.PDEPointResampler(period=100)

In [78]:
model = dde.Model(data, net)
model.compile("adam", lr=lr, loss_weights=loss_weights)

Compiling model...
Building feed-forward neural network...
'build' took 0.051663 s



'compile' took 0.809187 s



In [79]:
losshistory, train_state = model.train(display_every=100, iterations=n_adam, callbacks=[resampler])

Training model...

Step      Train loss                                                                          Test loss                                                                           Test metric
0         [3.69e-01, 3.26e+02, 0.00e+00, 0.00e+00, 0.00e+00, 8.40e-02, 2.88e-02, 3.49e+00]    [3.69e-01, 3.26e+02, 0.00e+00, 0.00e+00, 0.00e+00, 8.40e-02, 2.88e-02, 3.49e+00]    []  
100       [1.69e-01, 6.22e-02, 1.68e-02, 3.48e-04, 1.89e-03, 3.63e-02, 3.07e-03, 4.40e-02]    [1.69e-01, 6.22e-02, 1.68e-02, 3.48e-04, 1.89e-03, 3.63e-02, 3.07e-03, 4.40e-02]    []  
200       [4.86e-02, 1.08e-02, 1.64e-03, 1.29e-05, 1.97e-03, 4.92e-03, 7.64e-03, 8.35e-03]    [4.86e-02, 1.08e-02, 1.64e-03, 1.29e-05, 1.97e-03, 4.92e-03, 7.64e-03, 8.35e-03]    []  
300       [2.59e-02, 7.13e-03, 3.80e-04, 2.34e-07, 7.29e-04, 1.37e-03, 6.69e-03, 3.13e-03]    [2.59e-02, 7.13e-03, 3.80e-04, 2.34e-07, 7.29e-04, 1.37e-03, 6.69e-03, 3.13e-03]    []  
400       [1.43e-02, 5.29e-03, 1.33e-04, 7.45e-06, 2.44e-04